In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Implement me
# Set working directory
abspath = '/content/drive/My Drive/Colab Notebooks/NLP/NLP-Final-Project/NLP Project'

Mounted at /content/drive


In [2]:
import os
os.chdir(abspath)
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/NLP/NLP-Final-Project/NLP Project'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import dateutil.parser

In [2]:
# dictionary of text dates and links (August 2016 to November 2016)

date1 = '2016-08-10' 
url1 = 'https://www.reuters.com/article/us-samsung-elec-m-a-dacor-idUSKCN10L2RR'

date2 = '2016-08-22'
url2 = 'https://www.reuters.com/article/us-samsung-elec-phones-refurbishment-idUSKCN10X0FT'

date3 = '2016-08-24'
url3 = 'https://www.reuters.com/article/us-samsung-elec-tencent-holdings-stocks-idUSKCN10Z0BJ'

date4 = '2016-08-31'
url4 = 'https://www.reuters.com/article/us-nokia-corp-managementchanges-idUSKCN1160I8'


date5 = '2016-09-02'
url5 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-replacement-idUSKCN1180TV'

date6 = '2016-09-06'
url6 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKCN11C0CV'

date7 = '2016-09-11'
url7 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-stocks-idUSKCN11I02M'

date8 = '2016-09-15'
url8 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKCN11L2M1'

date7 = '2016-09-19'
url7 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-china-idUSKCN11P0J9'

date8 = '2016-09-30'
url8 = 'https://www.reuters.com/article/us-samsung-china-idUSKCN1200LD'

date9 = '2016-11-03'
url9 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKBN12Y0T7'


date10 = '2016-11-05'
url10 = 'https://www.reuters.com/article/us-samsung-elec-smartphones-idUSKBN13101Q'

date11 = '2016-11-14'
url11 = 'https://www.reuters.com/article/us-harman-intl-ind-samsung-elec-idUSKBN1390O9'

date12 = '2016-11-20'
url12 = 'https://www.reuters.com/article/us-samsung-elec-recall-brand-idUSKBN13F146'

date13 = '2016-11-22'
url13 = 'https://www.reuters.com/article/us-southkorea-politics-idUSKBN13I049'

date14 = '2016-11-28'
url14 = 'https://www.reuters.com/article/us-samsung-elec-restructuring-idUSKBN13N2GS'



url_dict = {date1:url1,
            date2:url2,
            date3:url3,
            date4:url4,
            date5:url5,
            date6:url6,
            date7:url7,
            date8:url8,
            date9:url9,
            date10:url10,
            date11:url11,
            date12:url12,
            date13:url13,
            date14:url14,
           }


In [3]:
# extract info from each website and store it in dataframe
df_text = pd.DataFrame(columns = ['Date','Header','Text'])
header = []
text = []
date = []
for key in url_dict:
    page = requests.get(url_dict[key]).text
    soup = BeautifulSoup(page, 'html.parser')
    date.append(datetime.strptime(key, '%Y-%m-%d').date())
    url_header = soup.find('h1').text
    header.append(url_header)
    url_text = soup.find('div', class_ ='ArticleBodyWrapper')
    url_text = url_text.text
    text.append(url_text)
df_text['Date']=date
df_text['Header']=header
df_text['Text']=text

# remove the author and "min read" sections
df_text_temp = [] 
for text in df_text['Text']:
    loc = text.find('(Reuters)')
    df_text_temp.append(text[loc+9:])
df_text['Text'] = df_text_temp




In [4]:
df_text['Date'] = df_text['Date'].apply(lambda x: pd.to_datetime(x))
df_text.head()

,Date,Header,Text
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi..."
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...


In [5]:
# import stock data of netflix
df_stock = pd.read_csv('Samsung.csv', header=0)

df_stock['Date']=df_stock['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


df_stock = df_stock.drop(['High','Low','Close','Adj Close'], axis=1)
df_stock.head()

,Date,Open,Volume
0,2016-01-04,25200.0,15346950
1,2016-01-05,24040.0,10800100
2,2016-01-06,24160.0,18337600
3,2016-01-07,23320.0,14119400
4,2016-01-08,23260.0,12888150


In [6]:
# Merge two dataframes
df = pd.merge(df_text, df_stock, how='inner', on='Date')
df = df.sort_values('Date')

In [7]:
# show % change in stock price after news article publish
pct_stock = df['Open'].pct_change()*100
df['% Change in Stock Price'] = pct_stock
pct_volume = df['Volume'].pct_change()*100
df['% Change in Volume'] = pct_volume
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639


# Processing the text


## import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [8]:
# import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [9]:

text_nlp = nlp(df['Text'][1]) 
# Remove stop words
text_nlp_clean = [word for word in text_nlp if word.is_stop == False]
# Lemmatize words
text_nlp_clean = [word.lemma_ for word in text_nlp_clean]

In [10]:
text_clean = []
for ind, row in df.iterrows():
    
    # lower casing
    text_lower = row['Text'].lower()
    
    # tokenization
    text_nlp = nlp(row['Text']) 
    
    # remove punctuation and empty space
    text_nlp_clean = [word for word in text_nlp if not word.is_punct | word.is_space]
    
    # Remove stop words
    text_nlp_clean = [word for word in text_nlp_clean if word.is_stop == False]
    
    # Lemmatize tokens
    text_nlp_clean = [word.lemma_ for word in text_nlp_clean]
    
    # add preprocessed text to list
    text_clean.append(text_nlp_clean)

# turn list of strings to string
to_string = []
for text in text_clean:
    to_string.append(' '.join(text))


df['Text (clean)'] = to_string
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean)
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...


# Using a Pretrained Model 

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [12]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
# a function to return sentiment score
def polarity_score(text):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(text)['compound']

vader_score = []
for text in df['Text (clean)']:
    score = polarity_score(text)
    vader_score.append(score)

df['Vader Sentiment Score'] = vader_score

In [14]:
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score
0,2016-08-10,Samsung Electronics says to acquire U.S. appli...,- South Korean tech giant Samsung Electronics...,31340.0,12340350,NaN,NaN,south korean tech giant Samsung Electronics Co...,0.8555
1,2016-08-22,Samsung plans refurbished smartphone program: ...,- Samsung Electronics Co Ltd plans to launch ...,33480.0,12612200,6.828334,2.202936,Samsung Electronics Co Ltd plan launch program...,0.9971
2,2016-08-24,"Samsung, Tencent surge in race to become Asia'...",- Tencent Holdings Ltd 0700.HK and Samsung El...,33600.0,15938550,0.358423,26.374066,Tencent Holdings Ltd 0700.hk Samsung Electroni...,0.9963
3,2016-08-31,Nokia's patent chief departs in wake of Samsun...,"- The head of Nokia’s small, but highly profi...",32820.0,20532200,-2.321429,28.821003,head Nokia small highly profitable patent lice...,0.9930
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after bat...,- Samsung Electronics Co Ltd has recalled all...,31900.0,8153000,-2.803169,-60.291639,Samsung Electronics Co Ltd recall Galaxy Note ...,0.9854


## Sentiment Analysis

In [15]:
# run this cell to import nltk
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from utils import process_tweet, build_freqs

In [16]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:

# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [18]:
# split the data into two pieces (80-20), one for training and one for testing (validation set)  
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [19]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [20]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [21]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11340


In [22]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [23]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

In [24]:
Y.shape

(8000, 1)

### SVM Parameters

In [25]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, Y)

C:\Users\aashi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(random_state=0)

In [26]:

# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [27]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet_prob(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict_proba(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [28]:

my_tweet = 'The plot was terrible and I was sad until the ending!'
predict_tweet(my_tweet, freqs)

array([0.])

# Testing the Model 

In [29]:
sentiment = []
sentiment_prob = []
for i in df.iloc[:,2]:
    sentiment.append(predict_tweet(i, freqs)[0])
    sentiment_prob.append(predict_tweet_prob(i, freqs)[0][1])

In [30]:
sentiment


[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [31]:
sentiment_prob

[0.6695515692647688,
 0.012659091349997681,
 0.8326791209409139,
 0.07580211607174459,
 0.0025454909101454495,
 0.018872802031474317,
 0.14695255146151398,
 0.22099345466835518,
 0.383674231977853,
 0.012820320610267732,
 0.01687993067573554,
 0.009441774949025096]

In [32]:
df['sentiment'] = sentiment
df['sentiment_prob'] = sentiment_prob


#DEEP LEARNING

In [35]:
import sklearn
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score, confusion_matrix
import sklearn.metrics
import tensorflow as tf
from scipy.special import softmax
import os
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import nltk
from tqdm import tqdm
import spacy
import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F
from scipy.special import softmax
from torchtext.vocab import Vocab
from nltk.corpus import twitter_samples 

nlp = spacy.load('en')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [37]:
# %% ----------------------------------- Hyper Parameters --------------------------------------------------------------
class Args:
    def __init__(self):
        self.seq_len = "get_max_from_data"
        self.embedding_dim = 50
        self.n_epochs = 10
        self.lr = 1e-2
        self.batch_size = 512
        self.train = True
        self.save_model = True


args = Args()

In [38]:
# %% ----------------------------------- Helper Functions --------------------------------------------------------------
def acc(x, y, return_labels=False):
    with torch.no_grad():
        logits = torch.empty(len(x), 2)
        for batch in range(len(x) // args.batch_size + 1):
            inds = slice(batch * args.batch_size, (batch + 1) * args.batch_size)
            logits[inds] = model(x[inds])
        pred_labels = np.argmax(logits.cpu().numpy(), axis=1)
    if return_labels:
        return pred_labels
    else:
        return 100*accuracy_score(y.cpu().numpy(), pred_labels)


def extract_vocab_dict_and_msl(sentences_train, sentences_dev):
    """ Tokenizes all the sentences and gets a dictionary of unique tokens and also the maximum sequence length """
    tokens, ms_len = [], 0
    for sentence in list(sentences_train) + list(sentences_dev):
        tokens_in_sentence = nltk.word_tokenize(sentence)
        if ms_len < len(tokens_in_sentence):
            ms_len = len(tokens_in_sentence)
        tokens += tokens_in_sentence
    token_vocab = {key: i for key, i in zip(set(tokens), range(1, len(set(tokens))+1))}
    if len(np.unique(list(token_vocab.values()))) != len(token_vocab):
        "There are some rep words..."
    return token_vocab, ms_len


def convert_to_ids(raw_sentences, vocab_dict, pad_to):
    """ Takes an NumPy array of raw text sentences and converts to a sequence of token ids """
    x = np.empty((len(raw_sentences), pad_to))
    for idx, sentence in enumerate(raw_sentences):
        word_ids = []
        for token in nltk.word_tokenize(sentence):

            if token not in vocab_dict.keys():
              continue 
            try:
                word_ids.append(vocab_dict[token])
            except:
                word_ids.append(vocab_dict[token])
        if pad_to < len(word_ids):
            x[idx] = word_ids[:pad_to]
        else:
            x[idx] = word_ids + [0] * (pad_to - len(word_ids))
    return x


def get_glove_embeddings(vocab_dict):
    with open("glove.6B.50d.txt", "r") as s:
        glove = s.read()
    embeddings_dict = {}
    for line in glove.split("\n")[:-1]:
        text = line.split()
        if text[0] in vocab_dict:
            embeddings_dict[vocab_dict[text[0]]] = torch.from_numpy(np.array(text[1:], dtype="float32"))
    return embeddings_dict


def get_glove_table(vocab_dict, glove_dict):
    lookup_table = torch.empty((len(vocab_dict)+2, 50))
    for token_id in sorted(vocab_dict.values()):
        if token_id in glove_dict:
            lookup_table[token_id] = glove_dict[token_id]
        else:
            lookup_table[token_id] = torch.zeros((1, 50))  # For unknown tokens
    lookup_table[0] = torch.zeros((1, 50))
    return lookup_table



In [39]:
data_train = pd.read_csv("original_data/train.tsv", sep="\t")
x_train_raw, y_train = data_train["sentence"].values, torch.LongTensor(data_train["label"].values).to(device)
data_dev = pd.read_csv("original_data/dev.tsv", sep="\t")
x_dev_raw, y_dev = data_dev["sentence"].values, torch.LongTensor(data_dev["label"].values).to(device)


In [40]:
# %% -------------------------------------- Data Prep ------------------------------------------------------------------
try:
    with open("example_prep_data/vocab_dict.json", "r") as s:
        token_ids = json.load(s)
    msl = np.load("example_prep_data/max_sequence_length.npy").item()
except:
    print("Tokenizing all the examples to get a vocab dict and the maximum sequence length...")
    token_ids, msl = extract_vocab_dict_and_msl(x_train_raw, x_dev_raw)
    os.mkdir("example_prep_data")
    with open("example_prep_data/vocab_dict.json", "w") as s:
        json.dump(token_ids, s)
    np.save("example_prep_data/max_sequence_length.npy", np.array([msl]))
if args.seq_len == "get_max_from_data":
    args.seq_len = msl

glove_embeddings = get_glove_embeddings(token_ids)

try:
    x_train = np.load("example_prep_data/prep_train_len{}.npy".format(args.seq_len))
    x_dev = np.load("example_prep_data/prep_dev_len{}.npy".format(args.seq_len))
except:
    print("Converting all the sentences to sequences of token ids...")
    x_train = convert_to_ids(x_train_raw, token_ids, args.seq_len)
    np.save("example_prep_data/prep_train_len{}.npy".format(args.seq_len), x_train)
    x_dev = convert_to_ids(x_dev_raw, token_ids, args.seq_len)
    np.save("example_prep_data/prep_dev_len{}.npy".format(args.seq_len), x_dev)




In [41]:
x_train, x_dev = torch.LongTensor(x_train).to(device), torch.LongTensor(x_dev).to(device)

In [42]:
# %% -------------------------------------- CNN Class ------------------------------------------------------------------
class CNN(nn.Module):
    def __init__(self, vocab_size):
        super(CNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size + 2, args.embedding_dim)

        self.conv1 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 9)
        self.convnorm1 = nn.BatchNorm1d(args.embedding_dim)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 9)
        self.convnorm2 = nn.BatchNorm1d(args.embedding_dim)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(args.embedding_dim, args.embedding_dim, 7)
        self.linear = nn.Linear(args.embedding_dim, 2)
        self.act = torch.relu

    def forward(self, x):
        # nn.Conv1d operates on the columns, each embedding dimension is considered as one channel
        x = self.embedding(x).permute(0, 2, 1)
        x = self.pool1(self.convnorm1(self.act(self.conv1(x))))
        x = self.pool2(self.convnorm2(self.act(self.conv2(x))))
        return self.linear(self.act(self.conv3(x)).reshape(-1, args.embedding_dim))



In [43]:
# %% -------------------------------------- Training Prep ----------------------------------------------------------
model = CNN(len(token_ids)).to(device)
look_up_table = get_glove_table(token_ids, glove_embeddings)
model.embedding.weight.data.copy_(look_up_table)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
criterion = nn.CrossEntropyLoss()

In [44]:
# %% -------------------------------------- Training Loop ----------------------------------------------------------
labels_ditrib = torch.unique(y_dev, return_counts=True)
print("The no information rate is {:.2f}".format(100*labels_ditrib[1].max().item()/len(y_dev)))
if args.train:
    acc_dev_best = 0
    print("Starting training loop...")
    for epoch in range(args.n_epochs):

        loss_train, train_steps = 0, 0
        model.train()
        total = len(x_train) // args.batch_size + 1  # Initiates a progress bar that will be updated for each batch
        with tqdm(total=total, desc="Epoch {}".format(epoch)) as pbar:  # "Epoch" will be updated for each epoch
            for batch in range(len(x_train)//args.batch_size + 1):
                inds = slice(batch*args.batch_size, (batch+1)*args.batch_size)
                optimizer.zero_grad()
                logits = model(x_train[inds])
                loss = criterion(logits, y_train[inds])
                loss.backward()
                optimizer.step()
                loss_train += loss.item()
                train_steps += 1
                pbar.update(1)  # Updates the progress and the training loss
                pbar.set_postfix_str("Training Loss: {:.5f}".format(loss_train / train_steps))

        model.eval()
        with torch.no_grad():
            y_dev_pred = model(x_dev)
            loss = criterion(y_dev_pred, y_dev)
            loss_test = loss.item()

        acc_dev = acc(x_dev, y_dev)
        print("Epoch {} | Train Loss {:.5f}, Train Acc {:.2f} - Test Loss {:.5f}, Test Acc {:.2f}".format(
            epoch, loss_train/train_steps, acc(x_train, y_train), loss_test, acc_dev))

        if acc_dev > acc_dev_best and args.save_model:
            torch.save(model.state_dict(), "cnn_sentiment.pt")
            print("The model has been saved!")
            acc_dev_best = acc_dev

Epoch 0:   0%|          | 0/132 [00:00<?, ?it/s]

The no information rate is 50.92
Starting training loop...


Epoch 0: 100%|██████████| 132/132 [00:01<00:00, 78.35it/s, Training Loss: 0.33921]


Epoch 0 | Train Loss 0.33921, Train Acc 93.69 - Test Loss 0.59996, Test Acc 80.16


Epoch 1:  15%|█▌        | 20/132 [00:00<00:01, 101.83it/s, Training Loss: 0.20040]

...

Epoch 7 | Train Loss 0.03465, Train Acc 98.37 - Test Loss 1.45630, Test Acc 77.75


Epoch 9:  14%|█▍        | 19/132 [00:00<00:01, 96.47it/s, Training Loss: 0.02912]

Epoch 8 | Train Loss 0.02927, Train Acc 99.21 - Test Loss 1.05540, Test Acc 78.78


Epoch 9: 100%|██████████| 132/132 [00:01<00:00, 99.35it/s, Training Loss: 0.02349]


Epoch 9 | Train Loss 0.02349, Train Acc 99.27 - Test Loss 1.33796, Test Acc 78.56


In [45]:

x = None
deep_learning_sentiment_prob = []

for i in df.iloc[:,7]:


  test = convert_to_ids([i.lower()], token_ids, args.seq_len)


  test = torch.LongTensor(test).to(device)
  pred = model(test).cpu().detach().numpy()[0]
  print(pred)

  x = softmax(pred)
  deep_learning_sentiment_prob.append(x[1])

[21.500319 -9.975771]
[17.522757 -7.729133]
[-8.09799    4.3018894]
[-0.04248523 -0.02919023]
[ 5.798468  -1.8659534]
[ 6.025967  -1.7648915]
[17.68392   -7.3724775]
[ 7.746011  -3.1824927]
[-9.330252   5.6907187]
[-0.39469644  0.23006952]
[ 0.8088293  -0.34689245]
[13.106791 -4.366239]


In [46]:
df['deep_learning_sentiment_prob'] = deep_learning_sentiment_prob
df

Date  ... deep_learning_sentiment_prob
0  2016-08-10  ...  2.138494e-14               
1  2016-08-22  ...  1.079552e-11               
2  2016-08-24  ...  9.999957e-01               
3  2016-08-31  ...  5.033237e-01               
4  2016-09-02  ...  4.690080e-04               
5  2016-09-06  ...  4.133267e-04               
6  2016-09-19  ...  1.312639e-11               
7  2016-09-30  ...  1.793922e-05               
8  2016-11-03  ...  9.999995e-01               
9  2016-11-14  ...  6.513017e-01               
10 2016-11-22  ...  2.394455e-01               
11 2016-11-28  ...  2.579642e-08               

[12 rows x 12 columns]

# Summary 

In [33]:


# Converting % stock price change to % stock price change per day
days = [0]
for i in range(len(df['Date'])):
    if i != 0:
        date_diff = df['Date'][i]-df['Date'][i-1]
        day = date_diff.days
        days.append(day)
df['Days Elapsed'] = days
df['Average % Change in Stock Price'] = df['% Change in Stock Price']/df['Days Elapsed']
df['Average % Change in Volume'] = df['% Change in Volume']/df['Days Elapsed']

df['Logistic Regression Sentiment'] = sentiment
df['Logistic Regression Sentiment Score'] = sentiment_prob


df.columns



Index(['Date', 'Header', 'Text', 'Open', 'Volume', '% Change in Stock Price',
       '% Change in Volume', 'Text (clean)', 'Vader Sentiment Score',
       'sentiment', 'sentiment_prob', 'Days Elapsed',
       'Average % Change in Stock Price', 'Average % Change in Volume',
       'Logistic Regression Sentiment', 'Logistic Regression Sentiment Score'],
      dtype='object')

## Outlier Removal

In [34]:
Q1_stock = df['% Change in Stock Price'].quantile(0.25)
Q3_stock = df['% Change in Stock Price'].quantile(0.75)
IQR_stock = Q3_stock - Q1_stock
lower_bound_stock = Q1_stock-1.5*IQR_stock
upper_bound_stock = Q3_stock+1.5*IQR_stock

Q1_volume = df['% Change in Volume'].quantile(0.25)
Q3_volume = df['% Change in Volume'].quantile(0.75)
IQR_volume = Q3_volume - Q1_volume
lower_bound_volume = Q1_volume-1.5*IQR_volume
upper_bound_volume = Q3_volume+1.5*IQR_volume




df_plot_stock = df[(df['% Change in Stock Price']>lower_bound_stock) & (df['% Change in Stock Price']<upper_bound_stock)]
df_plot_volume = df[(df['% Change in Volume']>lower_bound_volume) & (df['% Change in Volume']<upper_bound_stock)]

df_plot_stock.columns
print(np.corrcoef(df_plot_stock['Average % Change in Stock Price'], df_plot_stock['Vader Sentiment Score'])[1,0])
print(np.corrcoef(df_plot_stock['Average % Change in Stock Price'], df_plot_stock['Logistic Regression Sentiment Score'])[1,0])
print(np.corrcoef(df_plot_volume['Average % Change in Volume'], df_plot_volume['Vader Sentiment Score'])[1,0])
print(np.corrcoef(df_plot_volume['Average % Change in Volume'], df_plot_volume['Logistic Regression Sentiment Score'])[1,0])


0.03326779029559743
0.1793908117821089
-0.347293515802899
0.3444833412586505


In [48]:
df['Vader Sentiment'] = df['Vader Sentiment Score'].apply(lambda x: "positive" if x>0 else "negative")
df['Logistic Regression Sentiment'] = df['sentiment_prob'].apply(lambda x: "positive" if x>0.5 else "negative")
df['Stock Price Change'] = df['% Change in Stock Price Per Day'].apply(lambda x: "positive" if x>0 else "negative")
df['deep_learning_sentiment_prob'] = df['deep_learning_sentiment_prob'].apply(lambda x: "positive" if x>0.5 else "negative")

In [49]:
columns = ['Date','Header','Stock Price Change', 'Vader Sentiment', 'Logistic Regression Sentiment','deep_learning_sentiment_prob']
df_summary = df[columns]
df_summary.head()

,Date,Header,Stock Price Change,Vader Sentiment,Logistic Regression Sentiment,deep_learning_sentiment_prob
0,2016-08-10,Samsung Electronics says to acquire U.S. appliances maker Dacor,negative,positive,positive,negative
1,2016-08-22,Samsung plans refurbished smartphone program: source,positive,positive,negative,negative
2,2016-08-24,"Samsung, Tencent surge in race to become Asia's most valuable firm",positive,positive,positive,positive
3,2016-08-31,Nokia's patent chief departs in wake of Samsung pacts,negative,positive,negative,positive
4,2016-09-02,Samsung recalls Galaxy Note 7 phones after battery fires,negative,positive,negative,negative
